<a href="https://colab.research.google.com/github/Bora-Ulu/Inflation-Technique/blob/main/Inflation_Technique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from numpy import *
from itertools import *
import time
from scipy.sparse import coo_matrix
from scipy.sparse import csc_matrix
from scipy.sparse import csr_matrix
from cvxopt import matrix, solvers, sparse, spmatrix
from numba import njit

@njit
def PrepForIntToArray32(ar):# Takes a huge column vector as input
                     # Returns a matrix where the rows are the base-4 binary expansions of each element of the input array
    #return transpose(atleast_2d(ravel(ar.astype(uint32)))).view(uint8)
    return reshape(ar.astype(uint32).view(uint8),(-1,4))


def IntToArray32(ar):
    return unpackbits(PrepForIntToArray32(ar),axis=1, bitorder='little').view(bool_)

#@njit
#def PrepForArrayToInt32(bitarray):
#    as2mat=bitarray.reshape(-1, bitarray.shape[-1])
#    temparray=zeros((len(as2mat),32),bool_)
#    temparray[:,:bitarray.shape[-1]]=as2mat
#    # return pad(bitarray.reshape(-1, bitarray.shape[-1]),[(0,0), (0, 32-bitarray.shape[-1])], mode='constant')
#    return temparray


def ArrayToInt32(bitarray):
    return packbits(bitarray, bitorder='little').view(uint32)

@njit
def ExtractBitsFromBitArray(bitarray,bitstoextract): #bitstoextract must be numpy array
    tempbitarray=zeros_like(bitarray)
    tempbitarray[:,:len(bitstoextract)]=bitarray[:,subtract(amax(bitstoextract),bitstoextract)]
    return tempbitarray

def ExtractBitsFromInts(intarray,bitstoextract): #bitstoextract need not be a numpy array
#    asbitarray=IntToArray32(intarray)
#    tempbitarray=zeros_like(asbitarray)
#    tempbitarray[:,:len(bitstoextract)]=asbitarray[:,subtract(amax(bitstoextract),array(bitstoextract))]
    return ArrayToInt32(ExtractBitsFromBitArray(IntToArray32(intarray),array(bitstoextract,uint16)))


ColumnIntegers=arange(0,4**12,1,uint32)
ColumnIntegersAsBits=IntToArray32(ColumnIntegers)
[ColumnIntegers.shape,ColumnIntegersAsBits.shape]

[(16777216,), (16777216, 32)]

In [2]:
arange(240,260).view(uint8).shape

(80,)

In [3]:
@njit
def PositionIndex(arraywithduplicates):
    arraycopy=zeros_like(arraywithduplicates)
    u=unique(arraywithduplicates)
    arraycopy[u]=arange(len(u))
    return arraycopy[arraywithduplicates]

PositionIndex(array([1,1,2,4,6,4,2,5,7],int32))

array([0, 0, 1, 2, 4, 2, 1, 3, 5])

In [4]:
RowIntegers=arange(0,4**6,1,uint32)
EncodingMonomialToRow=PositionIndex(amin(vstack((RowIntegers,ExtractBitsFromInts(RowIntegers,[2,3,0,1,6,7,4,5,10,11,8,9]))),axis=0))
print(EncodingMonomialToRow.shape)
EncodingMonomialToRow

(4096,)


array([   0,    1,    2, ..., 2069, 2043, 2079], dtype=uint32)

In [5]:
EncodingColumnToMonomial=ArrayToInt32(ExtractBitsFromBitArray(ColumnIntegersAsBits,array([0,1,6,7,8,9,14,15,16,17,22,23],uint16)))
print(EncodingColumnToMonomial.shape)
EncodingColumnToMonomial

(16777216,)


array([   0, 2048, 1024, ..., 3071, 2047, 4095], dtype=uint32)

In [6]:
multiplier=hstack((2**arange(24),zeros(8,uint32)))
SwapX1X2=arange(32);
SwapX1X2[23-array([0,1,2,3,4,5,6,7,8,9,12,13,10,11,14,15])]=SwapX1X2[23-array([2,3,0,1,6,7,4,5,12,13,8,9,14,15,10,11])];
SwapY1Y2=arange(32);
SwapY1Y2[23-array([0,1,4,5,2,3,6,7,16,17,18,19,20,21,22,23])]=SwapY1Y2[23-array([4,5,0,1,6,7,2,3,18,19,16,17,22,23,20,21])];
SwapZ1Z2=arange(32);
SwapZ1Z2[23-array([8,9,10,11,12,13,14,15,16,17,20,21,18,19,22,23])]=SwapZ1Z2[23-array([10,11,8,9,14,15,12,13,20,21,16,17,22,23,18,19])]
SymMultiplier=vstack((multiplier,\
     multiplier[SwapX1X2],\
     multiplier[SwapY1Y2], \
     multiplier[SwapZ1Z2], \
     multiplier[SwapX1X2[SwapY1Y2]], \
     multiplier[SwapX1X2[SwapZ1Z2]], \
     multiplier[SwapY1Y2[SwapZ1Z2]], \
     multiplier[SwapX1X2[SwapY1Y2[SwapZ1Z2]]],\
    ))

In [7]:
#TIMING COMPARISONS: Trimmed matrix multiplication
start = time.time()
A=dot(SymMultiplier[:,:24],ColumnIntegersAsBits.T[:24,:]);
print('It took', time.time()-start, 'seconds.')
A.shape

It took 3.7609951496124268 seconds.


(8, 16777216)

In [8]:
#TIMING COMPARISONS: Untrimmed matrix multiplication
# start = time.time()
# A=dot(ColumnIntegersAsBits,SymMultiplier.T).T;
# print('It took', time.time()-start, 'seconds.')
# A.shape

In [9]:
#TIMING COMPARISONS: Bit reshuffling and repacking
# start = time.time()
# A=vstack((ColumnIntegers,\
#     packbits(ColumnIntegersAsBits[:,SwapX1X2], bitorder='little').view(uint32),\
#     packbits(ColumnIntegersAsBits[:,SwapY1Y2], bitorder='little').view(uint32),\
#     packbits(ColumnIntegersAsBits[:,SwapZ1Z2], bitorder='little').view(uint32),\
#     packbits(ColumnIntegersAsBits[:,SwapX1X2[SwapY1Y2]], bitorder='little').view(uint32),\
#     packbits(ColumnIntegersAsBits[:,SwapX1X2[SwapZ1Z2]], bitorder='little').view(uint32),\
#     packbits(ColumnIntegersAsBits[:,SwapY1Y2[SwapZ1Z2]], bitorder='little').view(uint32),\
#     packbits(ColumnIntegersAsBits[:,SwapX1X2[SwapY1Y2[SwapZ1Z2]]], bitorder='little').view(uint32),\
#    ))
#print('It took', time.time()-start, 'seconds.')
#A.shape

In [10]:
#Removing duplicate columns
A=A[:,unique(amin(A,axis=0))]
A.shape

(8, 2123776)

In [11]:
EncodedA=EncodingMonomialToRow[EncodingColumnToMonomial][A]
#EncodedA=EncodingColumnToMonomial[A]
print(amax(EncodedA))
EncodedA.shape

2079


(8, 2123776)

EDITED UP TO THIS POINT. Next we need to construct the sparse matrix from this data, and generate the b vector from the distribution --- USING EncodingMonomialToRow !

In [12]:
EncodedA[:,:40]

array([[ 0,  4,  8, 12,  4,  8, 12,  8, 12, 12,  4,  8, 12,  0,  4,  8,
        12,  0,  4,  8, 12,  0,  4,  8, 12,  8, 12,  4,  8, 12,  0,  4,
         8, 12,  0,  4,  8, 12, 12,  4],
       [ 0,  4,  8, 12,  4,  8, 12,  8, 12, 12,  4,  8, 12,  0,  4,  8,
        12,  0,  4,  8, 12,  0,  4,  8, 12,  8, 12,  4,  8, 12,  0,  4,
         8, 12,  0,  4,  8, 12, 12,  4],
       [ 0,  0,  0,  0,  4,  4,  4,  8,  8, 12,  1,  1,  1,  5,  5,  5,
         5,  9,  9,  9,  9, 13, 13, 13, 13,  2,  2,  6,  6,  6, 10, 10,
        10, 10, 14, 14, 14, 14,  3,  7],
       [ 0,  0,  0,  0,  1,  1,  1,  2,  2,  3,  4,  4,  4,  5,  5,  5,
         5,  6,  6,  6,  6,  7,  7,  7,  7,  8,  8,  9,  9,  9, 10, 10,
        10, 10, 11, 11, 11, 11, 12, 13],
       [ 0,  0,  0,  0,  4,  4,  4,  8,  8, 12,  1,  1,  1,  5,  5,  5,
         5,  9,  9,  9,  9, 13, 13, 13, 13,  2,  2,  6,  6,  6, 10, 10,
        10, 10, 14, 14, 14, 14,  3,  7],
       [ 0,  0,  0,  0,  1,  1,  1,  2,  2,  3,  4,  4,  4,  5,  5,  5,
   

In [13]:
#columncount=EncodedA.shape[-1]
#columnspec=ravel(broadcast_to(arange(columncount), (8, columncount)))
#rowspec=ravel(broadcast_to(arange(8), (columncount,8)).T)
#rowspec
#ravel(EncodedA)

def FormSciPyArrayFromOnesPositions(OnesPositions):
    columncount=OnesPositions.shape[-1]
    columnspec=ravel(broadcast_to(arange(columncount), (8, columncount)))
    return coo_matrix((ones(OnesPositions.size,uint8), (ravel(EncodedA), columnspec)),(amax(OnesPositions)+1, columncount),dtype=uint8)
    
MSciPy=FormSciPyArrayFromOnesPositions(EncodedA)
MSciPy

<2080x2123776 sparse matrix of type '<class 'numpy.uint8'>'
	with 16990208 stored elements in COOrdinate format>

In [14]:
def FormCVXOPTArrayFromOnesPositions(OnesPositions):
    columncount=OnesPositions.shape[-1]
    columnspec=ravel(broadcast_to(arange(columncount), (8, columncount)))
    return spmatrix(ones(OnesPositions.size), ravel(EncodedA).tolist(), columnspec.tolist(),(amax(OnesPositions)+1, columncount))

MCVXOPT=FormCVXOPTArrayFromOnesPositions(EncodedA).T
MCVXOPT

<2123776x2080 sparse matrix, tc='d', nnz=16496028>

In [15]:
Data=[0.12199995751046305, 0.0022969343799089472, 0.001748319476328954, 3.999015242496535e-05, 0.028907881434196828, 0.0005736087488455967, 0.0003924033706699725, 1.1247230369521505e-05, 0.0030142577390317635, 0.09234476010282468, 4.373922921480586e-05, 0.0014533921021948346, 0.0007798079722868244, 0.024091567451515063, 1.1247230369521505e-05, 0.0003849052170902915, 0.020774884184769502, 0.000396152447459813, 0.0003049249122403608, 4.998769053120669e-06, 0.10820335492385, 0.0020794879260981982, 0.0015546171755205281, 2.4993845265603346e-05, 0.0006260958239033638, 0.020273757587194154, 7.498153579681003e-06, 0.0003374169110856452, 0.0028942872817568676, 0.08976414557915113, 2.624353752888351e-05, 0.0012984302615480939, 0.002370666223442477, 4.7488306004646356e-05, 0.0999928767540993, 0.001957018084296742, 0.0006198473625869629, 8.747845842961171e-06, 0.02636975644747481, 0.0005198719815245496, 1.4996307159362007e-05, 0.000403650601039494, 0.0005498645958432735, 0.017359475229224805, 7.123245900696953e-05, 0.002346922070440154, 0.0033754188031197316, 0.10295964618712641, 0.00038740460161685187, 7.498153579681003e-06, 0.01608353942841575, 0.000306174604503641, 0.0021319750011559654, 4.248953695152569e-05, 0.09107007399427891, 0.001860791780024169, 5.998522863744803e-05, 0.0018395470115484063, 0.002570616985567304, 0.0766411271224461, 1.874538394920251e-05, 0.00048238121362614454, 0.0006410921310627258, 0.020223769896662948]
preb=array(Data)
preb=kron(preb,preb)
preb=ravel(transpose(reshape(preb,(4,4,4,4,4,4)),(0,3,1,4,2,5)))

def MergeMonomials(bvector,encoding):
    return ravel(coo_matrix((bvector, (zeros(len(bvector),uint8), encoding)),(1, amax(encoding)+1)).toarray())

b=MergeMonomials(preb,EncodingMonomialToRow);
b.shape

(2080,)

In [16]:
MCVXOPT.size

(2123776, 2080)

In [17]:
#Proof of concept that spmatrix also sums duplicate entries, no need to go via SciPy
row  = array([0, 0, 1, 3, 1, 0, 0])
col  = array([0, 2, 1, 3, 1, 0, 0])
data = array([1, 1, 1, 1, 1, 1, 1])
coo = spmatrix(data, row.tolist(), col.tolist(), (4, 4))
print(-coo)

[-3.00e+00     0     -1.00e+00     0    ]
[    0     -2.00e+00     0         0    ]
[    0         0         0         0    ]
[    0         0         0     -1.00e+00]



In [18]:
import mosek
from cvxopt import msk
rowcount=MCVXOPT.size[0];
colcount=MCVXOPT.size[1];
CVXOPTb=matrix(atleast_2d(b).T)
CVXOPTh=matrix(zeros((rowcount,1)))
CVXOPTA=matrix(ones((1,colcount)))
solvers.lp(CVXOPTb,-MCVXOPT,CVXOPTh,CVXOPTA,matrix(ones((1,1))),solver='mosek')

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 2123777         
  Cones                  : 0               
  Scalar variables       : 2080            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.27            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 3.92    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization 

{'status': 'optimal',
 'x': <2080x1 matrix, tc='d'>,
 's': <2123776x1 matrix, tc='d'>,
 'y': <1x1 matrix, tc='d'>,
 'z': <2123776x1 matrix, tc='d'>,
 'primal objective': -1.9506798565794375e-05,
 'dual objective': -1.9506798604887057e-05,
 'gap': 1.2090503497545257e-13,
 'relative gap': 6.198097271966624e-09,
 'primal infeasibility': 2.3159252293680765e-11,
 'dual infeasibility': 0.0005902077338705597,
 'residual as primal infeasibility certificate': None,
 'residual as dual infeasibility certificate': None,
 'primal slack': -3.6155888314781886e-12,
 'dual slack': -1.8737150662485965e-06}